#### Azure Batch Shipyard devito 

In [1]:
# Allow multiple displays per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
DOTENV_FILE_PATH = ['.', 'not_shared'] + ['general.env']
SIBLING_DOCKER_DOTENV_FILE_PATH = ['.', 'not_shared'] + ['sibling_docker.env']

docker_file_location = 'docker_files'
docker_build_location = 'docker_build'
docker_build_no_cache = ''  # '--no-cache' # or '' #

control_plane_docker_image_name = "azure_batch_shipyard_devito"
control_plane_docker_image_tag = ""

azure_batch_shipyard_devito_base_image_name = 'devito_azure_base:v1.0'
azure_batch_shipyard_devito_tti_image_name = 'devito_azure_tti:v1.0'

az_cli_container = "azure_cli_container01"
signed_in_az_cli_image = 'new_az_cli_image'


use_ACR_and_dockerhub = True

In [3]:
import sys, os, pathlib
import platform,  dotenv
import subprocess

In [4]:
platform.platform()
os.getcwd()

'Linux-4.15.0-1071-azure-x86_64-with-debian-10.1'

'/workspace/notebooks'

In [5]:
def create_dot_env_file(dotenv_file_path_list):
    created_dotenv_file_path = os.path.join(*(dotenv_file_path_list)) 
    os.makedirs(os.path.join(*(dotenv_file_path_list[:-1])), exist_ok=True)
    pathlib.Path(created_dotenv_file_path).touch()
    return created_dotenv_file_path

dotenv_file_path = create_dot_env_file(DOTENV_FILE_PATH)
sibling_docker_dotenv_file_path = create_dot_env_file(SIBLING_DOCKER_DOTENV_FILE_PATH)

# # show .env file path
# !pwd
dotenv_file_path
sibling_docker_dotenv_file_path

'./not_shared/general.env'

'./not_shared/sibling_docker.env'

Save your sensitive info

In [6]:
ACR_NAME="fwi01acr"
ACR_PASSWORD=""
ACR_USERNAME="fwi01acr"

dockerhub_login = "georgedockeraccount"
dockerhub_pwd = ""

SUBSCRIPTION_ID=""

docker_container_mount_point = os.getcwd()

In [7]:
# dotenv.set_key(dotenv_file_path, 'ACR_NAME', ACR_NAME)
# dotenv.set_key(dotenv_file_path, 'ACR_PASSWORD', ACR_PASSWORD)
# dotenv.set_key(dotenv_file_path, 'ACR_USERNAME', ACR_USERNAME)

# dotenv.set_key(dotenv_file_path, 'DOCKERHUB_LOGIN', dockerhub_login)
# dotenv.set_key(dotenv_file_path, 'DOCKERHUB_PWD', dockerhub_pwd)

# dotenv.set_key(dotenv_file_path, 'SUBSCRIPTION_ID', SUBSCRIPTION_ID)

# dotenv.set_key(dotenv_file_path,'control_plane_docker_image_name', control_plane_docker_image_name)
# dotenv.set_key(dotenv_file_path,'control_plane_docker_image_tag', control_plane_docker_image_tag)

# dotenv.set_key(sibling_docker_dotenv_file_path, 'DOCKER_CONTAINER_MOUNT_POINT', docker_container_mount_point)

In [8]:
dotenv.load_dotenv(dotenv_path=dotenv_file_path)
docker_file_location = os.path.join(*(os.getcwd(), docker_file_location))

docker_file_name = 'Dockerfile'+ '_' + os.getenv('control_plane_docker_image_name')

docker_image_name_only = os.getenv('control_plane_docker_image_name')
image_version = os.getenv('control_plane_docker_image_tag')
if image_version!="":
    docker_file_name = docker_file_name +'_'+ image_version
    docker_image_name_only = docker_image_name_only +':'+ image_version

# docker_repo_name = os.getenv('DOCKERHUB_LOGIN') # or  
docker_repo_name = os.getenv('ACR_NAME')+'.azurecr.io' 
docker_image_name = docker_repo_name + '/' + docker_image_name_only

docker_file_dir = os.path.join(*([os.getcwd(), docker_file_location]))
os.makedirs(docker_file_dir, exist_ok=True)
docker_file_path = os.path.join(*([docker_file_dir]+[docker_file_name]))

docker_build_dir = os.path.join(*([os.getcwd(), docker_build_location]))
os.makedirs(docker_build_dir, exist_ok=True)


docker_image_name
docker_file_dir
docker_file_path
docker_build_dir

True

'fwi01acr.azurecr.io/azure_batch_shipyard_devito'

'/workspace/notebooks/docker_files'

'/workspace/notebooks/docker_files/Dockerfile_azure_batch_shipyard_devito'

'/workspace/notebooks/docker_build'

In [9]:
dotenv.load_dotenv(dotenv_path=sibling_docker_dotenv_file_path)
os.getenv('DOCKER_CONTAINER_MOUNT_POINT')

True

'/datadrive01/prj/Azure2019/notebooks'

##### Pull existing docker image from ACR, if available
[mcr.microsoft.com/azure-cli](https://github.com/Azure/azure-cli/blob/dev/Dockerfile) is based on alpine, so we need to install docker inside it

In [10]:
!docker rm -f $az_cli_container

azure_cli_container01


In [ ]:
cli_base_command='docker run '+ \
'-it '+ \
'--name ' + az_cli_container+ ' ' + \
'mcr.microsoft.com/azure-cli '+ \
'/bin/bash  -c ' 
internal_command = '"'+ \
'apk update && apk add --update --no-cache  openrc docker-cli docker; '+ \
'rc-update add docker boot; '+ \
' az login; '+ \
' az account set --subscription '+os.getenv('SUBSCRIPTION_ID')+ '; '+ \
' az account list -o table; '+\
'"'

cli_command = cli_base_command+internal_command

cli_command
!$cli_command

In [12]:
# mcr.microsoft.com/azure-cli container was sarted without rm option so it should show up as stopped
!docker container ls -a

CONTAINER ID        IMAGE                         COMMAND                  CREATED             STATUS                    PORTS                     NAMES
38f696dbccbd        mcr.microsoft.com/azure-cli   "/bin/bash -c 'apk u…"   32 seconds ago      Exited (0) 1 second ago                             azure_cli_container01
7afa1a2f11ae        continuumio/miniconda3        "/bin/bash -c 'expor…"   10 hours ago        Up 10 hours               0.0.0.0:10001->8888/tcp   miniconda3_container01


In [13]:
!docker commit $az_cli_container $signed_in_az_cli_image
!docker rm -f $az_cli_container

sha256:583e3244d51e7cc8174e6544ff51c1f6bd100575fbe2aec91f9fd3cb5f3b9393
azure_cli_container01


In [ ]:
cli_base_command=' docker run ' +\
'-it '+\
'--name ' + az_cli_container+ ' ' + \
'-v '+ os.getenv('DOCKER_CONTAINER_MOUNT_POINT') +'/../:/workspace:rw '+ \
'-v /var/run/docker.sock:/var/run/docker.sock '+ \
signed_in_az_cli_image+ ' '+\
'/bin/bash  -c ' 

internal_command = '"'+ \
' az account list -o table; '+\
'az acr login --name '+os.getenv('ACR_NAME')+ \
' --username '+os.getenv('ACR_USERNAME')+ \
' --password ' + os.getenv('ACR_PASSWORD')+'; '+\
'docker pull '+docker_image_name+'; '+\
'"'

cli_command = cli_base_command+internal_command

cli_command
!$cli_command

Dockerfile references:  
https://github.com/Azure/batch-shipyard/blob/master/images/docker/linux/cli/Dockerfile  
https://github.com/matsuu/docker-azure-cli/blob/master/Dockerfile  
https://github.com/Azure/azure-cli/blob/dev/Dockerfile  
https://github.com/Azure/azure-functions-core-tools#other-linux-distributions  

In [15]:
%%writefile $docker_file_path 

# https://hub.docker.com/r/microsoft/azure-cli/dockerfile   
FROM debian:latest 
MAINTAINER George Iordanescu <ghiordan@microsoft.com>

# https://github.com/Azure/batch-shipyard/blob/master/images/docker/linux/cli/Dockerfile
# https://github.com/Azure/batch-shipyard/blob/master/cargo/Dockerfile
RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends \
    python3 python3-pip python3-setuptools \
    ca-certificates curl apt-transport-https lsb-release gnupg \
    zip unzip \
    wget rsync git && \
    apt-get clean && \
    rm -rf /var/lib/apt/lists/*   && \
    git clone https://github.com/Azure/batch-shipyard.git /opt/batch-shipyard && \
    cd /opt/batch-shipyard && \
    rm -rf .git .github .vsts && \
    rm -f .git* .travis.yml *.yml install*  && \
    python3 -m pip install --no-cache-dir --upgrade pip  && \
    pip3 install --no-cache-dir -r requirements.txt  && \
    pip3 install --no-cache-dir --no-deps -r req_nodeps.txt && \
    python3 -m compileall -f /opt/batch-shipyard/shipyard.py /opt/batch-shipyard/convoy 
# ENTRYPOINT ["/opt/batch-shipyard/shipyard.py"]

# https://docs.microsoft.com/en-us/cli/azure/install-azure-cli-apt?view=azure-cli-latest#no-package-for-your-distribution
# ENV AZ_CLI_REPO=stretch   
RUN echo "deb [arch=amd64] https://packages.microsoft.com/repos/azure-cli/ $(lsb_release -sc) main" | \
    tee /etc/apt/sources.list.d/azure-cli.list && \
    curl -L https://packages.microsoft.com/keys/microsoft.asc | apt-key add - && \
    apt-get update && \
    apt-get install -y --no-install-recommends \
    azure-cli 

# https://github.com/Azure/azure-functions-core-tools
ENV AzFunctionsCoreTools_DIR /azure-functions-cli
RUN mkdir /tmp/AzFunctionsCoreTools3 && \
    mkdir -p $AzFunctionsCoreTools_DIR && \
    cd /tmp/AzFunctionsCoreTools3 && \
    wget --quiet --no-check-certificate \
    https://github.com/Azure/azure-functions-core-tools/releases/download/2.7.2100/Azure.Functions.Cli.linux-x64.2.7.2100.zip  && \
    unzip  -qq -d $AzFunctionsCoreTools_DIR Azure.Functions.Cli.linux-x64.*.zip && \
    rm -f Azure.Functions.Cli.linux-x64.*.zip && \
    cd $AzFunctionsCoreTools_DIR && \
    chmod +x func
ENV PATH=$AzFunctionsCoreTools_DIR:$PATH    

Overwriting /workspace/notebooks/docker_files/Dockerfile_azure_batch_shipyard_devito


In [16]:
cli_command='docker build -t '+ docker_image_name + \
' -f ' + docker_file_path + \
' ' + docker_build_dir + ' ' +\
docker_build_no_cache  + ' ' 
#  ' --target AzureBatchShipyardAzCliAzFunctionsCoreTools'

cli_command
# docker_build_response = ! $cli_command

# docker_build_response[0:5] 
# docker_build_response[-5:]

'docker build -t fwi01acr.azurecr.io/azure_batch_shipyard_devito -f /workspace/notebooks/docker_files/Dockerfile_azure_batch_shipyard_devito /workspace/notebooks/docker_build  '

In [17]:
! $cli_command

Sending build context to Docker daemon  3.631kB
Step 1/7 : FROM debian:latest
 ---> 971452c94376
Step 2/7 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Using cache
 ---> e21efbfcde33
Step 3/7 : RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends     python3 python3-pip python3-setuptools     ca-certificates curl apt-transport-https lsb-release gnupg     zip unzip     wget rsync git &&     apt-get clean &&     rm -rf /var/lib/apt/lists/*   &&     git clone https://github.com/Azure/batch-shipyard.git /opt/batch-shipyard &&     cd /opt/batch-shipyard &&     rm -rf .git .github .vsts &&     rm -f .git* .travis.yml *.yml install*  &&     python3 -m pip install --no-cache-dir --upgrade pip  &&     pip3 install --no-cache-dir -r requirements.txt  &&     pip3 install --no-cache-dir --no-deps -r req_nodeps.txt &&     python3 -m compileall -f /opt/batch-shipyard/shipyard.py /opt/batch-shipyard/convoy
 ---> Using cache
 ---> 8587cc01301c
Step 4/7 : RUN e

##### Build base docker image
 - use control plane docker image just built above
 - build azure_batch_shipyard_devito_base_image_name defined by src/AzureBatch/docker/base_image/Dockerfile
 - creates public and private ssh keys in /root/.ssh/ in azure_batch_shipyard_devito_base_image_name docker image
 - cp /root/.ssh/id_rsa.pub /root/.ssh/authorized_keys

In [18]:
cli_base_command='docker run '+ \
'-it '+ \
'--rm '+ \
'--name azure-cli_container01 '+ \
'-v '+ os.getenv('DOCKER_CONTAINER_MOUNT_POINT') +'/../:/workspace:rw '+ \
'-v /usr/bin/docker:/usr/bin/docker '+ \
'-v /var/run/docker.sock:/var/run/docker.sock '+ \
docker_image_name + ' '+ \
'/bin/bash -c ' 

#'-v ~/.ssh:/root/.ssh:ro '+ \
# cli_command = cli_base_command+'"pwd; ls -l /workspace"'

image_name = azure_batch_shipyard_devito_base_image_name
sibling_docker_build_command = '"cd /workspace/src/AzureBatch/docker/base_image/ &&  docker build -t '+\
image_name + ' ."'
cli_command = cli_base_command+sibling_docker_build_command
cli_command
!$cli_command

'docker run -it --rm --name azure-cli_container01 -v /datadrive01/prj/Azure2019/notebooks/../:/workspace:rw -v /usr/bin/docker:/usr/bin/docker -v /var/run/docker.sock:/var/run/docker.sock fwi01acr.azurecr.io/azure_batch_shipyard_devito /bin/bash -c "cd /workspace/src/AzureBatch/docker/base_image/ &&  docker build -t devito_azure_base:v1.0 ."'

Sending build context to Docker daemon  4.096kB
Step 1/9 : FROM ubuntu:18.04
 ---> 72300a873c2c
Step 2/9 : COPY ssh_config /root/.ssh/config
 ---> Using cache
 ---> 051d8c548634
Step 3/9 : RUN apt-get update     && apt-get install -y --no-install-recommends         build-essential ca-certificates wget openssh-client openssh-server         mpich libmpich-dev     && mkdir -p /var/run/sshd     && ssh-keygen -A     && sed -i 's/UsePAM yes/UsePAM no/g' /etc/ssh/sshd_config     && sed -i 's/#PermitRootLogin yes/PermitRootLogin yes/g' /etc/ssh/sshd_config     && sed -i 's/#RSAAuthentication yes/RSAAuthentication yes/g' /etc/ssh/sshd_config     && sed -i 's/#PubkeyAuthentication yes/PubkeyAuthentication yes/g' /etc/ssh/sshd_config     && ssh-keygen -f /root/.ssh/id_rsa -t rsa -N ''     && chmod 600 /root/.ssh/config     && chmod 700 /root/.ssh     && cp /root/.ssh/id_rsa.pub /root/.ssh/authorized_keys
 ---> Using cache
 ---> e258920ebe25
Step 4/9 : RUN apt-get install -y htop vim git-core
 ---

##### Build tti docker image
 - use azure_batch_shipyard_devito_base_image_name docker image just built above
 - build azure_batch_shipyard_devito_tti_image_name defined by src/AzureBatch/docker/tti_image/Dockerfile
 - uses /root/.ssh/authorized_keys created above in azure_batch_shipyard_devito_base_image_name docker image

In [19]:
cli_base_command='docker run '+ \
'-it '+ \
'--rm '+ \
'--name azure-cli_container01 '+ \
'-v '+ os.getenv('DOCKER_CONTAINER_MOUNT_POINT') +'/../:/workspace:rw '+ \
'-v /root/.ssh:/root/.ssh:ro '+ \
'-v /usr/bin/docker:/usr/bin/docker '+ \
'-v /var/run/docker.sock:/var/run/docker.sock '+ \
azure_batch_shipyard_devito_base_image_name + ' '+ \
'/bin/bash -c ' 

image_name = azure_batch_shipyard_devito_tti_image_name
sibling_docker_build_command = '"cd /workspace/src/AzureBatch/docker/tti_image/ &&  docker build -t '+\
image_name + ' ."'
cli_command = cli_base_command+sibling_docker_build_command
cli_command
!$cli_command

'docker run -it --rm --name azure-cli_container01 -v /datadrive01/prj/Azure2019/notebooks/../:/workspace:rw -v /root/.ssh:/root/.ssh:ro -v /usr/bin/docker:/usr/bin/docker -v /var/run/docker.sock:/var/run/docker.sock devito_azure_base:v1.0 /bin/bash -c "cd /workspace/src/AzureBatch/docker/tti_image/ &&  docker build -t devito_azure_tti:v1.0 ."'

Sending build context to Docker daemon  84.99kB
Step 1/6 : FROM devito_azure_base:v1.0
 ---> 7c54e28d6723
Step 2/6 : RUN pip3 install azure-storage-blob h5py segyio boto3
 ---> Using cache
 ---> 9c1ebc86f719
Step 3/6 : RUN pip3 install azure-storage
 ---> Using cache
 ---> ad5eeea026b1
Step 4/6 : ADD ./tti /app/tti
 ---> Using cache
 ---> 8411b78c6093
Step 5/6 : EXPOSE 23
 ---> Using cache
 ---> 303227c5b533
Step 6/6 : CMD ["/usr/sbin/sshd", "-D", "-p", "23"]
 ---> Using cache
 ---> 8b061fdd3ae7
Successfully built 8b061fdd3ae7
Successfully tagged devito_azure_tti:v1.0


##### Use control plane docker image to self push into ACR

In [ ]:
full_azure_batch_shipyard_devito_base_image_name=docker_repo_name+ '/' + azure_batch_shipyard_devito_base_image_name
full_azure_batch_shipyard_devito_tti_image_name=docker_repo_name+ '/' + azure_batch_shipyard_devito_tti_image_name

cli_base_command='docker run '+ \
'-it '+ \
'--rm '+ \
'--name azure-cli_container01 '+ \
'-v '+ os.getenv('DOCKER_CONTAINER_MOUNT_POINT') +'/../:/workspace:rw '+ \
'-v /var/run/docker.sock:/var/run/docker.sock '+ \
signed_in_az_cli_image + ' '+ \
'/bin/bash -c ' 


internal_command = '"'+ \
' az account list -o table; '+ \
'az acr login --name '+os.getenv('ACR_NAME')+ \
' --username '+os.getenv('ACR_USERNAME')+ \
' --password ' + os.getenv('ACR_PASSWORD')+'; '+\
'docker push '+docker_image_name+'; '+\
'docker tag '+ azure_batch_shipyard_devito_base_image_name+' '+ full_azure_batch_shipyard_devito_base_image_name+'; '+\
'docker tag '+ azure_batch_shipyard_devito_tti_image_name+' '+ full_azure_batch_shipyard_devito_tti_image_name+'; '+\
'docker push '+full_azure_batch_shipyard_devito_base_image_name+'; '+\
'docker push '+full_azure_batch_shipyard_devito_tti_image_name+'; '+\
'"'
cli_command = cli_base_command+internal_command
cli_command
!$cli_command

In [21]:
if use_ACR_and_dockerhub:
    control_plane_docker_image_dockerhub_name = os.getenv('DOCKERHUB_LOGIN')+ '/' + docker_image_name_only
    cli_command='docker tag '+ docker_image_name+' '+ control_plane_docker_image_dockerhub_name
    cli_command

    docker_pwd = os.getenv('DOCKERHUB_PWD')
    docker_login = os.getenv('DOCKERHUB_LOGIN')
    !docker login -u=$docker_login -p=$docker_pwd
    !docker push {control_plane_docker_image_dockerhub_name}
 

'docker tag fwi01acr.azurecr.io/azure_batch_shipyard_devito georgedockeraccount/azure_batch_shipyard_devito'

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /root/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
The push refers to repository [docker.io/georgedockeraccount/azure_batch_shipyard_devito]
An image does not exist locally with the tag: georgedockeraccount/azure_batch_shipyard_devito


In [22]:
# !docker image ls
# !docker tag fwi01acr.azurecr.io/azure_batch_shipyard_devito georgedockeraccount/azure_batch_shipyard_devito
# !docker image ls
# !docker push georgedockeraccount/azure_batch_shipyard_devito